<a href="https://colab.research.google.com/github/August-murr/Data_science_Demonstration/blob/main/SQL%20Data%20Cleaning%20and%20Preperation/data_preparation_in_SQL_with_big_querry.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction
 This Jupyter Notebook serves as a showcase of my SQL skills, focusing on data cleaning, retrieval, and data preparation for analysis, machine learning, and visualization tasks.

 The dataset originates from a supermarket and comprises various tables, including Sales Records, Item Codes, Category Codes, and more.

 You can access the dataset [here](https://www.kaggle.com/datasets/yapwh1208/supermarket-sales-data/data).

  To work with the data, I've downloaded it to my Google Drive and utilized BigQuery within this Jupyter Notebook to present the results of each query as a Pandas DataFrame. Additionally, I've leveraged specific queries to prepare the data for visualization in Tableau.

## Authenticating google drive

As the dataset is hosted on Google Drive, additional steps are required to authenticate access to my Google Drive account using relevant Google libraries.

In [1]:
# Import necessary libraries for authentication and data analysis
import os  # Operating system library
from google.cloud import bigquery  # Google BigQuery client library
import google.auth  # Google authentication library
import matplotlib.pyplot as plt  # Matplotlib library for data visualization

In [2]:
# Set the path to your Google Application Credentials JSON file
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/content/big_query_sql_demonstration_key.json"

In [3]:
# Create credentials with Drive & BigQuery API scopes.
# Both APIs must be enabled for your project before running this code.
credentials, project = google.auth.default(
    scopes=[
        "https://www.googleapis.com/auth/drive",# Google Drive API scope
        "https://www.googleapis.com/auth/bigquery", # Google BigQuery API scope
    ]
)

## helper functions

In [4]:
# Construct a BigQuery client object.
client = bigquery.Client(credentials=credentials, project=project)

In [5]:
def query_to_df(query):
    """
    Executes a query and returns the result as a Pandas DataFrame for easier manipulation.

    Args:
        query (str): The SQL query to execute.

    Returns:
        pandas.DataFrame: The result of the query in a DataFrame format.
    """
    return client.query(query).result().to_dataframe()

In [6]:
def get_schema(table):
    """
    Retrieves the schema of a BigQuery table.

    Args:
        table (str): The name of the BigQuery table.

    Returns:
        List[google.cloud.bigquery.schema.SchemaField]: The schema of the table.
    """
    return client.get_table(table).schema

## Data Preview

In [7]:
# Define the SQL query to retrieve a sample of the data.
preview_query = """
SELECT * FROM `sql-demonstration.Super_Market_sales.sales_record` LIMIT 10
"""

In [8]:
# Execute the query and display the first few rows of the result.
preview_query = query_to_df(preview_query)
preview_query.head()

,Date,Time,Item_Code,Quantity_Sold__kilo_,Unit_Selling_Price__RMB_kg_,Sale_or_Return,Discount__Yes_No_
0,2020-07-01,09:15:07.924000,102900005117056,0.396,7.6,sale,False
1,2020-07-01,09:17:27.295000,102900005115960,0.849,3.2,sale,False
2,2020-07-01,09:17:33.905000,102900005117056,0.409,7.6,sale,False
3,2020-07-01,09:19:45.450000,102900005115823,0.421,10.0,sale,False
4,2020-07-01,09:20:23.686000,102900005115908,0.539,8.0,sale,False


In [9]:
# Retrieve and display the schema of the edited sales record table.
sales_record_schema = get_schema("sql-demonstration.Super_Market_sales.sales_record")
sales_record_schema

[SchemaField('Date', 'DATE', 'NULLABLE', None, None, (), None),
 SchemaField('Time', 'TIME', 'NULLABLE', None, None, (), None),
 SchemaField('Item_Code', 'INTEGER', 'NULLABLE', None, None, (), None),
 SchemaField('Quantity_Sold__kilo_', 'FLOAT', 'NULLABLE', None, None, (), None),
 SchemaField('Unit_Selling_Price__RMB_kg_', 'FLOAT', 'NULLABLE', None, None, (), None),
 SchemaField('Sale_or_Return', 'STRING', 'NULLABLE', None, None, (), None),
 SchemaField('Discount__Yes_No_', 'BOOLEAN', 'NULLABLE', None, None, (), None)]

## Creating Copy of the table

First we create a copy of the dataset to edit and preserve the original.

In [11]:
# Define the SQL query to create a copy of the table.
create_copy = """
-- Copy a table to create a new one
CREATE OR REPLACE TABLE `sql-demonstration.Super_Market_sales.sales_record_edited` AS
SELECT *
FROM `sql-demonstration.Super_Market_sales.sales_record`;
"""

In [12]:
# Execute the query to create the copy.
copy_query = client.query(create_copy)

## Checking for missing Values

Next, we'll examine the dataset for the presence of null values.

In [13]:
null_count = """
SELECT
  COUNTIF(Date IS NULL) AS Null_Date,
  COUNTIF(Time IS NULL) AS Null_Time,
  COUNTIF(Item_Code IS NULL) AS Null_Item_Code,
  COUNTIF(Quantity_Sold__kilo_ IS NULL) AS Null_Quantity_Sold_kilo,
  COUNTIF(Unit_Selling_Price__RMB_kg_ IS NULL) AS Null_Unit_Selling_Price_RMB_kg,
  COUNTIF(Sale_or_Return IS NULL) AS Null_Sale_or_Return,
  COUNTIF(Discount__Yes_No_ IS NULL) AS Null_Discount
FROM
  `sql-demonstration.Super_Market_sales.sales_record_edited`;
"""

In [14]:
null_count_query = query_to_df(null_count)

In [15]:
null_count_query

,Null_Date,Null_Time,Null_Item_Code,Null_Quantity_Sold_kilo,Null_Unit_Selling_Price_RMB_kg,Null_Sale_or_Return,Null_Discount
0,0,0,0,0,0,0,0


Fortunately, it appears that there are no missing values in the dataset.

## Correcting corrupted data

As I was exploring the data, I noticed an anomaly in the 'Quantity_Sold__kilo_' column. It seems that some rows were inputted incorrectly, resulting in negative values.  that or the store has discovered some kind of exotic matter and defied physics.

In [16]:
# Define the SQL query to count the number of negative quantity values.
negative_quantity = """
SELECT COUNTIF(Quantity_Sold__kilo_ < 0) AS Negative_Quantity_Count
FROM `sql-demonstration.Super_Market_sales.sales_record_edited`;
"""

In [17]:
# Execute the query to count negative quantity values
negative_quantity_query = query_to_df(negative_quantity)

In [18]:
negative_quantity_query

,Negative_Quantity_Count
0,461


It seems like the negative values are actually disatisfied costumers returning their purchases, if it actually was corrupted inputs, we could resolve this by converting these negative values to positive ones with the query below

In [19]:
# Define the SQL query to update negative values to their absolute values.
change_to_positive = """
UPDATE `sql-demonstration.Super_Market_sales.sales_record_edited`
SET Quantity_Sold__kilo_ = ABS(Quantity_Sold__kilo_)
WHERE Quantity_Sold__kilo_ < 0;
"""

In [ ]:
# Execute the query to change negative values to positive values.
change_to_positive_query = client.query(change_to_positive)

## New column for total cost of  purchase

now we'll create a new column for total cost of each purchase.

In [20]:
total_cost = """
SELECT
  Quantity_Sold__kilo_ * Unit_Selling_Price__RMB_kg_ AS total_cost
FROM
  `sql-demonstration.Super_Market_sales.sales_record_edited`;
"""

In [21]:
total_cost_query = query_to_df(total_cost)

In [22]:
total_cost_query.head()

,total_cost
0,3.213
1,3.913
2,4.109
3,3.409
4,2.107


## Converting date to day of week

To provide additional context to the data, we'll utilize the 'Date' column to determine the day of the week for each recorded sale.

In [23]:
# Define the SQL query to convert the 'Date' column to 'day_of_week'.
date_to_weekday = """
SELECT
  Date,
  FORMAT_DATE('%A', DATE(Date)) AS day_of_week
FROM
  `sql-demonstration.Super_Market_sales.sales_record_edited`;
"""

In [24]:
# Execute the query to convert the 'Date' column to 'day_of_week'
date_to_weekday_query = query_to_df(date_to_weekday)

In [25]:
date_to_weekday_query.head()

,Date,day_of_week
0,2021-03-02,Tuesday
1,2021-03-02,Tuesday
2,2021-03-02,Tuesday
3,2021-03-02,Tuesday
4,2021-03-02,Tuesday


we can also use this query to add this new column to the dataset

In [26]:
# Define the SQL query to add the 'day_of_week' column and update it with day of the week values.
update_for_weekday = """
-- Step 1: Add a new column to the dataset
ALTER TABLE `sql-demonstration.Super_Market_sales.sales_record_edited`
ADD COLUMN day_of_week STRING;

-- Step 2: Update the new column with day of the week values
UPDATE `sql-demonstration.Super_Market_sales.sales_record_edited`
SET day_of_week = FORMAT_DATE('%A', DATE(Date));
"""

## Calculating Daily Revenue

In [27]:
#Calculate total_cost for each sale.
Daily_revenue = """
-- Step 1: Calculate total_cost
WITH TotalCost AS (
  SELECT
    Date,
    Quantity_Sold__kilo_,
    Unit_Selling_Price__RMB_kg_,
    Quantity_Sold__kilo_ * Unit_Selling_Price__RMB_kg_ AS total_cost
  FROM
    `sql-demonstration.Super_Market_sales.sales_record_edited`
)

-- Step 2: Calculate daily revenue
SELECT
  Date,
  SUM(total_cost) AS Total_Revenue
FROM
  TotalCost
GROUP BY
  Date
ORDER BY
  Date;
"""

In [28]:
# Execute the query to calculate daily revenue.
daily_revenue_query = query_to_df(Daily_revenue)

In [29]:
daily_revenue_query

,Date,Total_Revenue
0,2020-07-01,3472.0136
1,2020-07-02,3456.5429
2,2020-07-03,3384.7701
3,2020-07-04,4394.8624
4,2020-07-05,4471.5748
...,...,...
1080,2023-06-26,1507.4716
1081,2023-06-27,1760.2499
1082,2023-06-28,1823.0318
1083,2023-06-29,2198.9108


## Calculating monthly revenue

In [30]:
monthly_revenue = """
-- Step 1: Calculate total_cost
WITH TotalCost AS (
  SELECT
    Date,
    Quantity_Sold__kilo_,
    Unit_Selling_Price__RMB_kg_,
    Quantity_Sold__kilo_ * Unit_Selling_Price__RMB_kg_ AS total_cost
  FROM
    `sql-demonstration.Super_Market_sales.sales_record_edited`
)

-- Step 2: Calculate monthly revenue
SELECT
  EXTRACT(YEAR FROM Date) AS Year,
  EXTRACT(MONTH FROM Date) AS Month,
  SUM(total_cost) AS Total_Revenue
FROM
  TotalCost
GROUP BY
  Year, Month
ORDER BY
  Year, Month;
"""

In [31]:
monthly_revenue_query = query_to_df(monthly_revenue)

In [32]:
monthly_revenue_query.head()

,Year,Month,Total_Revenue
0,2020,7,122550.2935
1,2020,8,133930.3319
2,2020,9,106520.8334
3,2020,10,122685.2828
4,2020,11,84574.6920


Tableau Link: [Revenue](https://public.tableau.com/views/SuperMarketanalysis_16958432799070/weeklyrevenue?:language=en-US&publish=yes&:display_count=n&:origin=viz_share_link)

## Highest selling product for each month

We will also use another dataset called "item code and category" to retrieve the name of each product from it's code.

In [33]:
monthly_highest_selling_products = """
-- Step 1: Calculate total_cost
WITH TotalCost AS (
  SELECT
    Date,
    Item_Code,
    Quantity_Sold__kilo_,
    Unit_Selling_Price__RMB_kg_,
    Quantity_Sold__kilo_ * Unit_Selling_Price__RMB_kg_ AS total_cost
  FROM
    `sql-demonstration.Super_Market_sales.sales_record_edited`
)

-- Step 2: Extract year and month
, YearMonth AS (
  SELECT
    EXTRACT(YEAR FROM Date) AS Year,
    EXTRACT(MONTH FROM Date) AS Month,
    Item_Code,
    SUM(total_cost) AS Total_Revenue
  FROM
    TotalCost
  GROUP BY
    Year, Month, Item_Code
)

-- Step 3: Rank products by monthly sales
, RankedProducts AS (
  SELECT
    Year,
    Month,
    Item_Code,
    Total_Revenue,
    RANK() OVER (PARTITION BY Year, Month ORDER BY Total_Revenue DESC) AS Rank
  FROM
    YearMonth
)

-- Step 4: Join with item_code_and_category to get product names
SELECT
  r.Year,
  r.Month,
  r.Item_Code,
  ic.Item_Name AS Product_Name,
  r.Total_Revenue
FROM
  RankedProducts r
JOIN
  `sql-demonstration.Super_Market_sales.item_code_and_category` ic
ON
  r.Item_Code = ic.Item_Code
WHERE
  r.Rank = 1
ORDER BY
  r.Year, r.Month;
"""

In [34]:
monthly_highest_selling_products_query = query_to_df(monthly_highest_selling_products)

In [35]:
monthly_highest_selling_products_query.head()

,Year,Month,Item_Code,Product_Name,Total_Revenue
0,2020,7,102900005116714,Broccoli,11541.4562
1,2020,8,102900005117056,Paopaojiao (Jingpin),14888.3145
2,2020,9,102900005117056,Paopaojiao (Jingpin),10723.4260
3,2020,10,102900005117056,Paopaojiao (Jingpin),11319.9850
4,2020,11,102900005117056,Paopaojiao (Jingpin),8650.0618


Here's the Tableau Visualization link:[Top Selling Products](https://public.tableau.com/views/SuperMarketanalysis_16958432799070/Topsellingproductseachmonth?:language=en-US&publish=yes&:display_count=n&:origin=viz_share_link)

## Daily sales of a product over time.

In this example, we've selected Broccoli. We'll encapsulate the process as a function so that you can obtain daily sales data for any product within the dataset.

In [36]:
def get_daily_sale(product_of_interest):
  daily_sale_of_product = f"""
  -- Filter rows for "Product" and calculate daily sales
  WITH ProductSales AS (
    SELECT
      Date,
      SUM(Quantity_Sold__kilo_) AS Daily_Sales
    FROM
      `sql-demonstration.Super_Market_sales.sales_record_edited`
    WHERE
      Item_Code IN (
        SELECT
          Item_Code
        FROM
          `sql-demonstration.Super_Market_sales.item_code_and_category`
        WHERE
          Item_Name = "{product_of_interest}"
      )
    GROUP BY
      Date
  )

  -- Show daily sales of "Product"
  SELECT
    Date,
    Daily_Sales
  FROM
    ProductSales
  ORDER BY
    Date;
  """
  return query_to_df(daily_sale_of_product)

In [37]:
get_daily_sale("Broccoli")

,Date,Daily_Sales
0,2020-07-01,34.554
1,2020-07-02,30.109
2,2020-07-03,26.460
3,2020-07-04,42.815
4,2020-07-05,42.365
...,...,...
1071,2023-06-26,8.083
1072,2023-06-27,14.272
1073,2023-06-28,13.401
1074,2023-06-29,15.180


## Daily Price of a product over time

We're exclusively considering non-discounted prices. In this example, we'll maintain Broccoli as our reference. Similar to the previous task, we'll also create a Python function to retrieve the historical price of any product of your choice.

In [38]:
def get_daily_price(product):
  daily_price = f"""
  -- Filter rows for your product with non-discount prices and retrieve distinct Unit_Selling_Price__RMB_kg_
  SELECT DISTINCT
    Date,
    Unit_Selling_Price__RMB_kg_
  FROM
    `sql-demonstration.Super_Market_sales.sales_record_edited`
  WHERE
    Item_Code IN (
      SELECT
        Item_Code
      FROM
        `sql-demonstration.Super_Market_sales.item_code_and_category`
      WHERE
        Item_Name = "{product}"
    )
    AND Discount__Yes_No_ = false
  ORDER BY
    Date;
  """
  return query_to_df(daily_price)

In [39]:
get_daily_price("Broccoli")

,Date,Unit_Selling_Price__RMB_kg_
0,2020-07-01,14.0
1,2020-07-02,14.0
2,2020-07-03,14.0
3,2020-07-04,14.0
4,2020-07-05,14.0
...,...,...
1081,2023-06-26,14.0
1082,2023-06-27,12.0
1083,2023-06-28,12.0
1084,2023-06-29,11.2


## How does the daily sales volume of each product category compare over time?

In [40]:
daily_category_revenue_contribution= """
-- Calculate daily revenue per category
WITH CategoryRevenue AS (
  SELECT
    s.Date,
    ic.Category_Name,
    SUM(s.Quantity_Sold__kilo_ * s.Unit_Selling_Price__RMB_kg_) AS Daily_Revenue
  FROM
    `sql-demonstration.Super_Market_sales.sales_record_edited` AS s
  JOIN
    `sql-demonstration.Super_Market_sales.item_code_and_category` AS ic
  ON
    s.Item_Code = ic.Item_Code
  WHERE
    s.Discount__Yes_No_ = false
  GROUP BY
    s.Date, ic.Category_Name
)

-- Pivot the data for selected categories
SELECT
  Date,
  MAX(IF(Category_Name = 'Aquatic Tuberous Vegetables', Daily_Revenue, NULL)) AS Aquatic_Tuberous_Vegetables,
  MAX(IF(Category_Name = 'Cabbage', Daily_Revenue, NULL)) AS Cabbage,
  MAX(IF(Category_Name = 'Capsicum', Daily_Revenue, NULL)) AS Capsicum,
  MAX(IF(Category_Name = 'Edible Mushroom', Daily_Revenue, NULL)) AS Edible_Mushroom,
  MAX(IF(CAST(Category_Name AS STRING) LIKE '%Flower/Leaf%Vegetables%', Daily_Revenue, NULL)) AS Flower_Leaf_Vegetables,
  MAX(IF(Category_Name = 'Solanum', Daily_Revenue, NULL)) AS Solanum
FROM
  CategoryRevenue
GROUP BY
  Date
ORDER BY
  Date;
"""

in the cell above, in this line:
```
# MAX(IF(CAST(Category_Name AS STRING) LIKE '%Flower/Leaf%Vegetables%', Daily_Revenue, NULL)) AS Flower_Leaf_Vegetables
```
We opted to use 'LIKE' instead of the equal sign ('=') due to potential data corruption or incorrect input in the 'item code and category' table."


In [41]:
daily_category_revenue_contribution_query = query_to_df(daily_category_revenue_contribution)

In [42]:
daily_category_revenue_contribution_query.head(10)

,Date,Aquatic_Tuberous_Vegetables,Cabbage,Capsicum,Edible_Mushroom,Flower_Leaf_Vegetables,Solanum
0,2020-07-01,70.2838,592.530,759.9902,365.4820,1446.6496,176.818
1,2020-07-02,53.1208,546.032,669.7044,605.9290,1317.6876,172.092
2,2020-07-03,44.1000,495.368,670.6646,546.2950,1366.3026,196.243
3,2020-07-04,118.7532,702.186,820.5558,570.9072,1828.3128,327.403
4,2020-07-05,40.1900,697.982,1009.9126,732.6238,1544.1420,403.455
5,2020-07-06,27.0800,500.443,645.2908,484.2030,1434.6660,201.412
6,2020-07-07,31.1700,526.991,531.0704,615.1326,1258.3780,186.848
7,2020-07-08,155.4840,532.102,741.5910,565.4192,1129.7650,228.424
8,2020-07-09,21.0600,496.204,615.3550,464.1238,1395.6710,205.520
9,2020-07-10,125.0460,613.936,687.1768,406.6940,1166.6546,241.684


We used another query to prepare the data for the area chart that was created in tableau.

In [43]:
daily_category_revenue_for_area_chart = """
-- Calculate daily revenue per category
WITH CategoryRevenue AS (
  SELECT
    s.Date,
    ic.Category_Name,
    SUM(s.Quantity_Sold__kilo_ * s.Unit_Selling_Price__RMB_kg_) AS Daily_Revenue
  FROM
    `sql-demonstration.Super_Market_sales.sales_record_edited` AS s
  JOIN
    `sql-demonstration.Super_Market_sales.item_code_and_category` AS ic
  ON
    s.Item_Code = ic.Item_Code
  WHERE
    s.Discount__Yes_No_ = false
  GROUP BY
    s.Date, ic.Category_Name
)
-- Summarize daily revenue per category
SELECT
  Date,
  Category_Name,
  SUM(Daily_Revenue) AS Category_Daily_Revenue
FROM
  CategoryRevenue
GROUP BY
  Date, Category_Name
ORDER BY
  Date, Category_Name;
"""

In [44]:
daily_category_revenue_for_area_chart_query = query_to_df(daily_category_revenue_for_area_chart)

In [45]:
daily_category_revenue_for_area_chart_query

,Date,Category_Name,Category_Daily_Revenue
0,2020-07-01,Aquatic Tuberous Vegetables,70.2838
1,2020-07-01,Cabbage,592.5300
2,2020-07-01,Capsicum,759.9902
3,2020-07-01,Edible Mushroom,365.4820
4,2020-07-01,Flower/Leaf Vegetables,1446.6496
...,...,...,...
6468,2023-06-30,Cabbage,282.6200
6469,2023-06-30,Capsicum,489.7992
6470,2023-06-30,Edible Mushroom,236.8280
6471,2023-06-30,Flower/Leaf Vegetables,620.9988


Tableau Link: [Revenue by Category](https://public.tableau.com/views/SuperMarketanalysis_16958432799070/revenuebycategory?:language=en-US&publish=yes&:display_count=n&:origin=viz_share_link)